# Getting to know PySpark
>  In this chapter, you'll learn how Spark manages data and how can you read and write tables from Python.

- toc: true 
- badges: true
- comments: true
- author: Lucas Nunes
- categories: [Datacamp]
- image: images/datacamp/___

> Note: This is a summary of the course's chapter 1 exercises "Introduction to PySpark" at datacamp. <br>[Github repo](https://github.com/lnunesAI/Datacamp/) / [Course link](https://www.datacamp.com/tracks/machine-learning-scientist-with-python)

In [2]:
%%capture
!pip install pyspark
import pyspark
import pandas as pd
import numpy as np

## Introduction

### What is Spark, anyway?

<div class=""><p>Spark is a platform for cluster computing. Spark lets you spread data and computations over <em>clusters</em> with multiple <em>nodes</em> (think of each node as a separate computer). Splitting up your data makes it easier to work with very large datasets because each node only works with a small amount of data.</p>
<p>As each node works on its own subset of the total data, it also carries out a part of the total calculations required, so that both data processing and computation are performed <em>in parallel</em> over the nodes in the cluster. It is a fact that parallel computation can make certain types of programming tasks much faster.</p>
<p>However, with greater computing power comes greater complexity.</p>
<p>Deciding whether or not Spark is the best solution for your problem takes some experience, but you can consider questions like:</p>
<ul>
<li>Is my data too big to work with on a single machine?</li>
<li>Can my calculations be easily parallelized?</li>
</ul>

## Using Spark in Python

<div class=""><p>The first step in using Spark is connecting to a cluster.</p>
<p>In practice, the cluster will be hosted on a remote machine that's connected to all other nodes. There will be one computer, called the <em>master</em> that manages splitting up the data and the computations. The master is connected to the rest of the computers in the cluster, which are called <em>worker</em>. The master sends the workers data and calculations to run, and they send their results back to the master.</p>
<p>When you're just getting started with Spark it's simpler to just run a cluster locally. Thus, for this course, instead of connecting to another computer, all computations will be run on DataCamp's servers in a simulated cluster.</p>
<p>Creating the connection is as simple as creating an instance of the <code>SparkContext</code> class. The class constructor takes a few optional arguments that allow you to specify the attributes of the cluster you're connecting to.</p>
<p>An object holding all these attributes can be created with the <code>SparkConf()</code> constructor. Take a look at the <a href="http://spark.apache.org/docs/2.1.0/api/python/pyspark.html" target="_blank" rel="noopener noreferrer">documentation</a> for all the details!</p>
<p>For the rest of this course you'll have a <code>SparkContext</code> called <code>sc</code> already available in your workspace.</p>
<hr>
<p>How do you connect to a Spark cluster from PySpark?</p></div>

<pre>
Possible Answers
<b>Create an instance of the SparkContext class.</b>
Import the pyspark library.
Plug your computer into the cluster.
</pre>

### Examining The SparkContext

<div class=""><p>In this exercise you'll get familiar with the <code>SparkContext</code>.</p>
<p>You'll probably notice that code takes longer to run than you might expect. This is because Spark is some serious software. It takes more time to start up than you might be used to. You may also find that running simpler computations might take longer than expected. That's because all the optimizations that Spark has under its hood are designed for complicated operations with big data sets. That means that for simple or small problems Spark may actually perform worse than some other solutions!</p></div>

In [3]:
sc = pyspark.SparkContext()

Instructions
<div class="exercise--instructions__content"><p>Get to know the <code>SparkContext</code>.</p>
<ul>
<li>Call <code>print()</code> on <code>sc</code> to verify there's a <code>SparkContext</code> in your environment.</li>
<li><code>print()</code> <code>sc.version</code> to see what version of Spark is running on your cluster.</li>
</ul></div>

In [4]:
# Verify SparkContext
print(sc)

# Print Spark version
print(sc.version)

<SparkContext master=local[*] appName=pyspark-shell>
3.0.2


### Using DataFrames

<div class=""><p>Spark's core data structure is the Resilient Distributed Dataset (RDD). This is a low level object that lets Spark work its magic by splitting data across multiple nodes in the cluster. However, RDDs are hard to work with directly, so in this course you'll be using the Spark DataFrame abstraction built on top of RDDs.</p>
<p>The Spark DataFrame was designed to behave a lot like a SQL table (a table with variables in the columns and observations in the rows). Not only are they easier to understand, DataFrames are also more optimized for complicated operations than RDDs.</p>
<p>When you start modifying and combining columns and rows of data, there are many ways to arrive at the same result, but some often take much longer than others. When using RDDs, it's up to the data scientist to figure out the right way to optimize the query, but the DataFrame implementation has much of this optimization built in!</p>
<p>To start working with Spark DataFrames, you first have to create a <code>SparkSession</code> object from your <code>SparkContext</code>. You can think of the <code>SparkContext</code> as your connection to the cluster and the <code>SparkSession</code> as your interface with that connection.</p>
<p>Remember, for the rest of this course you'll have a <code>SparkSession</code> called <code>spark</code> available in your workspace!</p>
<hr>
<p>Which of the following is an advantage of Spark DataFrames over RDDs?</p></div>

<pre>
Possible Answers
<b>Operations using DataFrames are automatically optimized.</b>
They are smaller.
They can perform more kinds of operations.
They can hold more kinds of data.
</pre>

### Creating a SparkSession

<p>We've already created a <code>SparkSession</code> for you called <code>spark</code>, but what if you're not sure there already is one? Creating multiple <code>SparkSession</code>s and <code>SparkContext</code>s can cause issues, so it's best practice to use the <code>SparkSession.builder.getOrCreate()</code> method. This returns an existing <code>SparkSession</code> if there's already one in the environment, or creates a new one if necessary!</p>

Instructions
<ul>
<li>Import <code>SparkSession</code> from <code>pyspark.sql</code>.</li>
<li>Make a new <code>SparkSession</code> called <code>my_spark</code> using <code>SparkSession.builder.getOrCreate()</code>.</li>
<li>Print <code>my_spark</code> to the console to verify it's a <code>SparkSession</code>.</li>
</ul>

In [5]:
# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession

# Create my_spark
my_spark = SparkSession.builder.getOrCreate()

# Print my_spark
print(my_spark)

### Viewing tables

<div class=""><p>Once you've created a <code>SparkSession</code>, you can start poking around to see what data is in your cluster!</p>
<p>Your <code>SparkSession</code> has an attribute called <code>catalog</code> which lists all the data inside the cluster. This attribute has a few methods for extracting different pieces of information.</p>
<p>One of the most useful is the <code>.listTables()</code> method, which returns the names of all the tables in your cluster as a list.</p></div>

In [6]:
%%capture
!wget https://github.com/lnunesAI/Datacamp/raw/main/2-machine-learning-scientist-with-python/21-introduction-to-pyspark/datasets/flights_small.csv
file_path = "flights_small.csv"
spark = (SparkSession
  .builder
  .appName("flights")
  .getOrCreate())
flights = (spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load(file_path))
flights.createOrReplaceTempView("flights")

Instructions
<li>See what tables are in your cluster by calling <code>spark.catalog.listTables()</code> and printing the result!</li>

In [7]:
# Print the tables in the catalog
print(spark.catalog.listTables())

[Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


### Are you query-ious?

<div class=""><p>One of the advantages of the DataFrame interface is that you can run SQL queries on the tables in your Spark cluster. If you don't have any experience with SQL, don't worry, we'll provide you with queries! (To learn more SQL, start with our <a href="https://www.datacamp.com/courses/intro-to-sql-for-data-science" target="_blank" rel="noopener noreferrer">Introduction to SQL</a> course.)</p>
<p>As you saw in the last exercise, one of the tables in your cluster is the <code>flights</code> table. This table contains a row for every flight that left Portland International Airport (PDX) or Seattle-Tacoma International Airport (SEA) in 2014 and 2015.</p>
<p>Running a query on this table is as easy as using the <code>.sql()</code> method on your <code>SparkSession</code>. This method takes a string containing the query and returns a DataFrame with the results!</p>
<p>If you look closely, you'll notice that the table <code>flights</code> is only mentioned in the query, not as an argument to any of the methods. This is because there isn't a local object in your environment that holds that data, so it wouldn't make sense to pass the table as an argument.</p>
<p>Remember, we've already created a <code>SparkSession</code> called <code>spark</code> in your workspace. (It's no longer called <code>my_spark</code> because we created it for you!)</p></div>

Instructions
<ul>
<li>Use the <code>.sql()</code> method to get the first 10 rows of the <code>flights</code> table and save the result to <code>flights10</code>. The variable <code>query</code> contains the appropriate SQL query.</li>
<li>Use the DataFrame method <code>.show()</code> to print <code>flights10</code>.</li>
</ul>

In [8]:
# Don't change this query
query = "FROM flights SELECT * LIMIT 10"

# Get the first 10 rows of flights
flights10 = spark.sql(query)

# Show the results
flights10.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

### Pandafy a Spark DataFrame

<div class=""><p>Suppose you've run a query on your huge dataset and aggregated it down to something a little more manageable.</p>
<p>Sometimes it makes sense to then take that table and work with it locally using a tool like <code>pandas</code>. Spark DataFrames make that easy with the <code>.toPandas()</code> method. Calling this method on a Spark DataFrame returns the corresponding <code>pandas</code> DataFrame. It's as simple as that!</p>
<p>This time the query counts the number of flights to each airport from SEA and PDX.</p>
<p>Remember, there's already a <code>SparkSession</code> called <code>spark</code> in your workspace!</p></div>

Instructions
<ul>
<li>Run the query using the <code>.sql()</code> method. Save the result in <code>flight_counts</code>.</li>
<li>Use the <code>.toPandas()</code> method on <code>flight_counts</code> to create a <code>pandas</code> DataFrame called <code>pd_counts</code>.</li>
<li>Print the <code>.head()</code> of <code>pd_counts</code> to the console.</li>
</ul>

In [9]:
# Don't change this query
query = "SELECT origin, dest, COUNT(*) as N FROM flights GROUP BY origin, dest"

# Run the query
flight_counts = spark.sql(query)

# Convert the results to a pandas DataFrame
pd_counts = flight_counts.toPandas()

# Print the head of pd_counts
print(pd_counts.head())

  origin dest    N
0    SEA  RNO    8
1    SEA  DTW   98
2    SEA  CLE    2
3    SEA  LAX  450
4    PDX  SEA  144


### Put some Spark in your data

<div class=""><p>In the last exercise, you saw how to move data from Spark to <code>pandas</code>. However, maybe you want to go the other direction, and put a <code>pandas</code> DataFrame into a Spark cluster! The <code>SparkSession</code> class has a method for this as well.</p>
<p>The <code>.createDataFrame()</code> method takes a <code>pandas</code> DataFrame and returns a Spark DataFrame.</p>
<p>The output of this method is stored locally, not in the <code>SparkSession</code> catalog. This means that you can use all the Spark DataFrame methods on it, but you can't access the data in other contexts.</p>
<p>For example, a SQL query (using the <code>.sql()</code> method) that references your DataFrame will throw an error. To access the data in this way, you have to save it as a <em>temporary table</em>.</p>
<p>You can do this using the <code>.createTempView()</code> Spark DataFrame method, which takes  as its only argument the name of the temporary table you'd like to register. This method registers the DataFrame as a table in the catalog, but as this table is temporary, it can only be accessed from the specific <code>SparkSession</code> used to create the Spark DataFrame.</p>
<p>There is also the method <code>.createOrReplaceTempView()</code>. This safely creates a new temporary table if nothing was there before, or updates an existing table if one was already defined. You'll use this method to avoid running into problems with duplicate tables.</p>
<p>Check out the diagram to see all the different ways your Spark data structures interact with each other.</p>
<p><img src="https://s3.amazonaws.com/assets.datacamp.com/production/course_4452/datasets/spark_figure.png" alt=""></p>
<p>There's already a <code>SparkSession</code> called <code>spark</code> in your workspace, <code>numpy</code> has been imported as <code>np</code>, and <code>pandas</code> as <code>pd</code>.</p></div>

Instructions
<ul>
<li>The code to create a <code>pandas</code> DataFrame of random numbers has already been provided and saved under <code>pd_temp</code>.</li>
<li>Create a Spark DataFrame called <code>spark_temp</code> by calling the <code>.createDataFrame()</code> method with <code>pd_temp</code> as the argument.</li>
<li>Examine the list of tables in your Spark cluster and verify that the new DataFrame is <em>not</em> present. Remember you can use <code>spark.catalog.listTables()</code> to do so.</li>
<li>Register <code>spark_temp</code> as a temporary table named <code>"temp"</code> using the <code>.createOrReplaceTempView()</code> method. Remember that the table name is set including it as the only argument!</li>
<li>Examine the list of tables again!</li>
</ul>

In [10]:
# Create pd_temp
pd_temp = pd.DataFrame(np.random.random(10))

# Create spark_temp from pd_temp
spark_temp = spark.createDataFrame(pd_temp)

# Examine the tables in the catalog
print(spark_temp)

# Add spark_temp to the catalog
spark_temp.createOrReplaceTempView("temp")

# Examine the tables in the catalog again
print(spark.catalog.listTables())

DataFrame[0: double]
[Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


### Dropping the middle man

<div class=""><p>Now you know how to put data into Spark via <code>pandas</code>, but you're probably wondering why deal with <code>pandas</code> at all? Wouldn't it be easier to just read a text file straight into Spark? Of course it would!</p>
<p>Luckily, your <code>SparkSession</code> has a <code>.read</code> attribute which has several methods for reading different data sources into Spark DataFrames. Using these you can create a DataFrame from a .csv file just like with regular <code>pandas</code> DataFrames!</p>
<p>The variable <code>file_path</code> is a string with the path to the file <code>airports.csv</code>. This file contains information about different airports all over the world.</p>
<p>A <code>SparkSession</code> named <code>spark</code> is available in your workspace.</p></div>

In [11]:
%%capture
!wget https://github.com/lnunesAI/Datacamp/raw/main/2-machine-learning-scientist-with-python/21-introduction-to-pyspark/datasets/airports.csv

Instructions
<ul>
<li>Use the <code>.read.csv()</code> method to create a Spark DataFrame called <code>airports</code><ul>
<li>The first argument is <code>file_path</code></li>
<li>Pass the argument <code>header=True</code> so that Spark knows to take the column names from the first line of the file.</li></ul></li>
<li>Print out this DataFrame by calling <code>.show()</code>.</li>
</ul>

In [12]:
# Don't change this file path
file_path = "airports.csv"

# Read in the airports data
airports = spark.read.csv(file_path, header=True)

# Show the data
airports.show()

+---+--------------------+----------------+-----------------+----+---+---+
|faa|                name|             lat|              lon| alt| tz|dst|
+---+--------------------+----------------+-----------------+----+---+---+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|
|0S9|Jefferson County ...|      48.0538086|     -122.8106436| 108| -8|  A|
|0W3|Harford County Ai...